In [1]:
import random
random.sample(range(1,11),5)

[10, 7, 2, 6, 4]

In [4]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import glob
import datetime
#import pylabb
import matplotlib as plt
import datetime as datetime  
import statsmodels.api as sm  
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
%matplotlib inline

In [6]:
df = pd.read_csv('/Users/Petrichor/Desktop/all_student_data.csv', low_memory=False)

In [7]:
raw = df.copy()
raw.head(1)

,Unnamed: 0,B1,B2,B3,B4,B5,B7,MNDWI,NDVI,analyst,date,dfoID,id,lat,lon,random,strata,validation,score
0,0,675.522719,1128.643606,2015.265677,2436.991734,3702.040962,2753.348602,-0.365915,0.095263,HT,5/10/18,2143,0,-21.762688,123.87866,0.341315,0,0,0


In [8]:
raw.columns

Index([u'Unnamed: 0', u'B1', u'B2', u'B3', u'B4', u'B5', u'B7', u'MNDWI',
       u'NDVI', u'analyst', u'date', u'dfoID', u'id', u'lat', u'lon',
       u'random', u'strata', u'validation', u'score'],
      dtype='object')

In [15]:
list(set(raw.dfoID))

[4163, 2180, 2024, 2473, 4427, 3667, 2584, 2586, 4507, 2045, 3070, 2143]

### Loopin'!

In [78]:
floods = list(set(raw.dfoID)) ## Get a list of all the unique values of floods
sampling_levels = [10,20,30,40,50,60,100] ### Set levels you want to sample at 
col_names =  ['Flood','NumofPoints', 'tp', 'tn','fp','fn',
             'precision','recall','overall_accuracy',
             'comission','omission'] ### Column names for making data frames for each flood
my_df  = pd.DataFrame(columns = col_names)## Make empty data frame to put values for each sampling level in

for i in floods:  ### Loop through floods
    df_full = raw.loc[raw['dfoID']== i] ### subset data frame by single flood
    for j in sampling_levels: ## Loop through sampling levels
        df = df_full.sample(j) ##Apparently pandas has a built-in dataframe random sampler!
        tp = float(len(df.loc[df['score']== 5])) 
        tn = float(len(df.loc[df['score']== 0]))
        fp = float(len(df.loc[df['score']== 4]))
        fn = float(len(df.loc[df['score']== 1]))
        ### Calculating values!
        if tp == 0: ###Hmm, sometimes there are no tp scores, so you get a zero divide error
            continue ###I've put this in for now but it's gonna bite your butt eventually
        recall  = tp / (tp + fn)  # % of flooded pixels it ID'ed correctly
        accuracy = (tn + tp) / (tn+tp+fp+fn)
        commission = tp / (tp + fn)  #  modeled positives over all true positives
        omission = tp / (tp + fp)
        ##add a new row to the dataframe based on this sampling level!
        my_df.loc[len(my_df)] = [i,j, tp, tn, fp, fn, precision, recall, accuracy, commission, omission]
###Now you should have a full dataframe with all your sampling levels for each flood
###SO
my_df.to_csv('myprecious.csv', encoding='utf-8')